# EM dataset

In [ ]:
%matplotlib widget

In [ ]:
from pathlib import Path

import numpy as np
from lotr import DATASET_LOCATION
from lotr import plotting as pltltr
from lotr.analysis.pool_cell_info import get_pooled_cell_info
from lotr.em.core import MIDLINES
from lotr.em.loading import load_skeletons_from_zip
from matplotlib import pyplot as plt

COLS = pltltr.COLS

## Data loading

In [ ]:
# Reference:
# Load the gad1b stack and the annotation from the IPN atlas:

from bg_atlasapi.core import Atlas

atlas = Atlas(DATASET_LOCATION / "anatomy" / "ipn_zfish_0.5um_v1.8")

gad1b_stack = atlas.additional_references["gal4_gad1b"]


# Enlarge a bit the mask for the IPN (in the future we'll draw better masks):
mask = np.zeros((3, 3, 3)).astype(bool)
mask[:, 1, :] = True
annotation = (
    atlas.get_structure_mask("ipn") > 0
)  # binary_dilation(, iterations=7, structure=mask )
annotation_dipn = (
    atlas.get_structure_mask("dipn") > 0
)  # binary_dilation(, iterations=7, structure=mask)

In [ ]:
# Load the EM and mirror neurons on one side:
em_path = DATASET_LOCATION / "anatomy" / "em_reconstructions" / "aHB_dIPN.k.zip"

sel_neurons = load_skeletons_from_zip(em_path)

for n in sel_neurons:
    n.mirror_right = True

sel_neurons_not_mirr = load_skeletons_from_zip(em_path)


In [ ]:
#  Specify axes limits over all dimensions:
bs = dict(frontal=(30, 180), vertical=(-4, 170), sagittal=(-50, 120))

plotter = pltltr.AtlasPlotter(
    atlas=atlas,
    structures=["ipn", "dipn"],
    mask_slices=dict(frontal=slice(0, 120)),
    bounds_dict=dict(
        frontal=[bs["vertical"], bs["frontal"]],
        horizontal=[bs["sagittal"], bs["frontal"]],
        sagittal=[bs["vertical"], bs["sagittal"]],
    ),
)

## All projections with anatomy

In [ ]:
from scipy.ndimage import gaussian_filter

In [ ]:
cells_df = get_pooled_cell_info()

In [ ]:
coords_not_mirr = cells_df.loc[
    cells_df["hdn"], [f"centered{i}" for i in range(3)]
].values
coords_not_mirr[:, 1] += np.random.rand(coords_not_mirr.shape[0]) * 10

coords = coords_not_mirr.copy()
# Mirror on both sides

density_stack = np.zeros(gad1b_stack.shape)

left = coords[:, 2] < MIDLINES["ipn"]
coords[left, 2] = MIDLINES["ipn"] + (MIDLINES["ipn"] - coords[left, 2])
idx_coords = (coords * 2).astype(int)
density_stack[idx_coords[:, 0], idx_coords[:, 1], idx_coords[:, 2]] = 1

coords[:, 2] = MIDLINES["ipn"] - (coords[:, 2] - MIDLINES["ipn"])
idx_coords = (coords * 2).astype(int)
density_stack[idx_coords[:, 0], idx_coords[:, 1], idx_coords[:, 2]] = 1

In [ ]:
filt_dens = gaussian_filter(density_stack, 10)

In [ ]:
thr = 0.001
filt_dens_bin = filt_dens > thr  # [filt_dens < 0.0004] = 0
filt_dens_bin_l = filt_dens_bin.copy()
filt_dens_bin_l[:, :, : MIDLINES["ipn"] * 2] = 0
filt_dens_bin_r = filt_dens_bin.copy()
filt_dens_bin_r[:, :, MIDLINES["ipn"] * 2 :] = 0

In [ ]:
filt_dens_bin_l.shape

In [ ]:
plt.figure()
plt.imshow(np.max(filt_dens_bin_l, 1))

In [ ]:
%gui qt5

In [ ]:
import napari
from bg_atlasapi import BrainGlobeAtlas

In [ ]:
atlas = BrainGlobeAtlas("ipn_zfish_0.5um")

In [ ]:
v = napari.view_labels(filt_dens_bin_l, name="filt_dens_bin_l")
v.add_labels(filt_dens_bin_r, name="filt_dens_bin_r")
v.add_labels((atlas.annotation > 0).astype(int), name="ipn")

In [ ]:
import flammkuchen as fl

temp_backup_f = Path() / "ahb_ring.h5"
dict_to_save = {"ahb_ring_l": filt_dens_bin_l, "ahb_ring_r": filt_dens_bin_r}
fl.save(temp_backup_f, dict_to_save)